In [69]:
ENV["LINES"] = 10;
ENV["COLUMNS"] = 80;

HTML("""
<style>
.reveal pre code {
    max-height: none;
    font-size: 90%;
}
</style>
""")

HTML{String}("<style>\n.reveal pre code {\n    max-height: none;\n    font-size: 90%;\n}\n</style>\n")

# Julia + Jupyter + GPU = ⚗️🔬🧬🥰

Marius Millea (Project Scientist @ UC Davis in Cosmology)

NERSC GPU Science Day, Oct 12, 2023

## Outline
* Motivation
* Julia CUDA Installation
* Basic and advanced Julia CUDA usage
* Multi-GPU workflows for embarrasingly parallel problems

## Motivation

* Julia
    * interactive but fast
    * powerful and flexible
    * less boilerplate: code looks like science

* Jupyter
    * convenient for interactive work
    * fast iterative development workflow

* GPU
    * duh

## Install

Julia/CUDA install is drop-dead simple. Julia's CUDA package provides compatible binary drivers:

```shell
$ curl -fsSL https://install.julialang.org | sh
$ julia
pkg> add CUDA # ~2min
   Resolving package versions...
   Installed CUDA_Driver_jll ── v0.6.0+3
   Installed LLVMExtra_jll ──── v0.0.26+0
   ...
   Installed CUDA ───────────── v5.0.0
 Downloading artifact: CUDA_Driver
```

(Easy to select CUDA version _per project_ with e.g. `CUDA.set_runtime_version!(v"11.4")`)

I recommend this fully native Julia install over using any `modules`, i.e. I don't even have the `gpu` module loaded:

In [80]:
; module list


Currently Loaded Modules:
  1) craype-x86-milan                        8) cray-mpich/8.1.25
  2) libfabric/1.15.2.0                      9) craype/2.7.20
  3) craype-network-ofi                     10) gcc/11.2.0
  4) xpmem/2.6.2-2.5_2.27__gd067c3f.shasta  11) perftools-base/23.03.0
  5) PrgEnv-gnu/8.3.3                       12) cpe/23.03
  6) cray-dsmml/0.2.2                       13) xalt/2.10.2
  7) cray-libsci/23.02.1.1                  14) cray-python/3.9.13.1   (dev)

  Where:
   dev:  Development Tools and Programming Languages

 



This has proven robust across many clusters I've tried.

Checking everything is installed:

In [3]:
using CUDA

In [4]:
CUDA.versioninfo()

CUDA runtime 12.2, artifact installation
CUDA driver 12.2
NVIDIA driver 525.105.17, originally for CUDA 12.0

CUDA libraries: 
- CUBLAS: 12.2.5
- CURAND: 10.3.3
- CUFFT: 11.0.8
- CUSOLVER: 11.5.2
- CUSPARSE: 12.1.2
- CUPTI: 20.0.0
- NVML: 12.0.0+525.105.17

Julia packages: 
- CUDA: 5.0.0
- CUDA_Driver_jll: 0.6.0+3
- CUDA_Runtime_jll: 0.9.2+0

Toolchain:
- Julia: 1.9.3
- LLVM: 14.0.6
- PTX ISA support: 3.2, 4.0, 4.1, 4.2, 4.3, 5.0, 6.0, 6.1, 6.3, 6.4, 6.5, 7.0, 7.1, 7.2, 7.3, 7.4, 7.5
- Device capability support: sm_37, sm_50, sm_52, sm_53, sm_60, sm_61, sm_62, sm_70, sm_72, sm_75, sm_80, sm_86

4 devices:
  0: NVIDIA A100-SXM4-40GB (sm_80, 39.389 GiB / 40.000 GiB available)
  1: NVIDIA A100-SXM4-40GB (sm_80, 39.389 GiB / 40.000 GiB available)
  2: NVIDIA A100-SXM4-40GB (sm_80, 39.389 GiB / 40.000 GiB available)
  3: NVIDIA A100-SXM4-40GB (sm_80, 39.389 GiB / 40.000 GiB available)


## Basic usage

In [8]:
arr = rand(10_000_000)

10000000-element Vector{Float64}:
 0.9086257270453094
 0.09199119500242203
 0.5405112272332471
 ⋮
 0.1508900262074172
 0.9024278712197097

In [9]:
carr = cu(arr)

10000000-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 0.9086257
 0.09199119
 0.54051125
 ⋮
 0.15089002
 0.90242785

In [10]:
sin.(carr) .+ 1

10000000-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.7886596
 1.0918615
 1.5145744
 ⋮
 1.1503181
 1.7848338

Lets benchmark:

In [12]:
using BenchmarkTools

In [13]:
@btime CUDA.@sync sin.(carr) .+ 1;

  83.481 μs (41 allocations: 2.00 KiB)


In [14]:
@btime sin.(arr) .+ 1;

  68.553 ms (6 allocations: 76.29 MiB)


In [15]:
CUDA.@profile sin.(carr) .+ 1;

Profiler ran for 300.17 µs, capturing 11 events.

Host-side activity: calling CUDA APIs took 56.98 µs (18.98% of the trace)
┌──────────┬──────────┬───────┬──────────┬──────────┬──────────┬─────────────────────────┐
│ Time (%) │     Time │ Calls │ Avg time │ Min time │ Max time │ Name                    │
├──────────┼──────────┼───────┼──────────┼──────────┼──────────┼─────────────────────────┤
│    9.69% │ 29.09 µs │     1 │ 29.09 µs │ 29.09 µs │ 29.09 µs │ cuLaunchKernel          │
│    7.39% │ 22.17 µs │     1 │ 22.17 µs │ 22.17 µs │ 22.17 µs │ cuMemAllocFromPoolAsync │
└──────────┴──────────┴───────┴──────────┴──────────┴──────────┴─────────────────────────┘

Device-side activity: GPU was busy for 80.59 µs (26.85% of the trace)
┌──────────┬──────────┬───────┬──────────┬──────────┬──────────┬───────────────────────────────────────────────────────────────────────────────────────────────────────────────
│ Time (%) │     Time │ Calls │ Avg time │ Min time │ Max time │ Name              

## Power of Julia (1)

In Julia, you can easily put many arbitrary objects on GPU:

In [70]:
struct Point{T}
    x :: T
    y :: T
end

In [71]:
arr = Point.(rand(100), rand(100))
carr = cu(arr)

100-element CuArray{Point{Float64}, 1, CUDA.Mem.DeviceBuffer}:
 Point{Float64}(0.9036834915690581, 0.7280460429572848)
 Point{Float64}(0.7708203673107145, 0.05294668526958213)
 Point{Float64}(0.5394529633521908, 0.8032629228332351)
 ⋮
 Point{Float64}(0.6624703153358394, 0.4720296666892744)
 Point{Float64}(0.8116771423344411, 0.8097130787459793)

In e.g. Jax/PyTorch/TF, the only things you can stick inside of CUDA arrays are Int/Float/Complex. In Julia, anything with a static memory layout is fine.

In [72]:
distance_from_origin(p::Point) = sqrt(p.x^2 + p.y^2)

distance_from_origin (generic function with 1 method)

In [73]:
distance_from_origin.(carr)

100-element CuArray{Float64, 1, CUDA.Mem.DeviceBuffer}:
 1.1604718409337662
 0.7726366482002138
 0.9675953817934705
 ⋮
 0.8134364910280052
 1.1464968614350834

## Limitations

In [20]:
function distance_from_origin_bad(p::Point)
    sqrt(sum([p.x^2, p.y^2]))
end

distance_from_origin_bad (generic function with 1 method)

In [21]:
distance_from_origin_bad.(carr)

LoadError: InvalidIRError: compiling MethodInstance for (::GPUArrays.var"#broadcast_kernel#32")(::CUDA.CuKernelContext, ::CuDeviceVector{Float64, 1}, ::Base.Broadcast.Broadcasted{CUDA.CuArrayStyle{1}, Tuple{Base.OneTo{Int64}}, typeof(distance_from_origin_bad), Tuple{Base.Broadcast.Extruded{CuDeviceVector{Point{Float64}, 1}, Tuple{Bool}, Tuple{Int64}}}}, ::Int64) resulted in invalid LLVM IR
[31mReason: unsupported call through a literal pointer[39m[31m (call to ijl_alloc_array_1d)[39m
Stacktrace:
  [1] [0m[1mArray[22m
[90m    @[39m [90m./[39m[90m[4mboot.jl:477[24m[39m
  [2] [0m[1mArray[22m
[90m    @[39m [90m./[39m[90m[4mboot.jl:486[24m[39m
  [3] [0m[1msimilar[22m
[90m    @[39m [90m./[39m[90m[4mabstractarray.jl:884[24m[39m
  [4] [0m[1msimilar[22m
[90m    @[39m [90m./[39m[90m[4mabstractarray.jl:883[24m[39m
  [5] [0m[1m_array_for[22m
[90m    @[39m [90m./[39m[90m[4marray.jl:671[24m[39m
  [6] [0m[1m_array_for[22m
[90m    @[39m 

Limitations on code in functions that will be compiled for GPU:
* No calls to CPU functions
   * E.g. creating Arrays (use StaticArrays.jl instead)
* No _dynamic dispatch_
   * Code should be _type stable_

## Power of Julia (2)

You can also directly write kernels in Julia, giving you the full power and flexibility of CUDA kernel programming:

In [22]:
function my_kernel(carr_out, carr)   
    start = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    stride = blockDim().x * gridDim().x
    len = length(carr)
    for i = start:stride:len  # "grid-stride" loop
        carr_out[i] = sin(carr[i]) + 1
    end
    return
end

my_kernel (generic function with 1 method)

In [23]:
carr = cu(rand(10_000_000))
carr_out = similar(carr);

In [24]:
@cuda threads=256 my_kernel(carr_out, carr)

CUDA.HostKernel for my_kernel(CuDeviceVector{Float32, 1}, CuDeviceVector{Float32, 1})

In [25]:
carr_out

10000000-element CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}:
 1.1289185
 1.385206
 1.44538
 ⋮
 1.8264189
 1.7915335

See [Kernel Programming](https://cuda.juliagpu.org/stable/api/kernel/) for full list of CUDA.jl kernel programming capabilities.

## Multi-GPU (single node)

In [26]:
CUDA.devices()

CUDA.DeviceIterator() for 4 devices:
0. NVIDIA A100-SXM4-40GB
1. NVIDIA A100-SXM4-40GB
2. NVIDIA A100-SXM4-40GB
3. NVIDIA A100-SXM4-40GB

In [27]:
CUDA.device()

CuDevice(0): NVIDIA A100-SXM4-40GB

In [28]:
CUDA.device!(1)

CuDevice(1): NVIDIA A100-SXM4-40GB

In [29]:
arr = rand(10_000_000)
carr = cu(arr)
@btime CUDA.@sync sin.(carr) .+ 1;

  84.943 μs (41 allocations: 2.00 KiB)


CUDA.jl does its own memory management, so before switching back to GPU 0, give back memory (don't usually have to think about this unless you use the same GPU from multiple processes, which for the purpose of this demo I do):

In [83]:
GC.gc()
CUDA.reclaim()

In [85]:
CUDA.device!(0)

CuDevice(0): NVIDIA A100-SXM4-40GB

You can use multiple GPUs via Julia processes, tasks, or threads. 

The most robust and easy way I have found (as of 2023), which I recommend starting with, is per-_process_:

In [32]:
using Distributed

In [33]:
addprocs(3)

3-element Vector{Int64}:
 2
 3
 4

In [34]:
@everywhere using CUDA, BenchmarkTools

In [35]:
@everywhere procs() println((myid(), CUDA.device()))

(1, CuDevice(0))
      From worker 2:	(2, CuDevice(0))
      From worker 4:	(4, CuDevice(0))
      From worker 3:	(3, CuDevice(0))


In [36]:
@everywhere procs() CUDA.device!(myid()-1)

In [37]:
@everywhere procs() println((myid(), CUDA.device()))

(1, CuDevice(0))
      From worker 4:	(4, CuDevice(3))
      From worker 3:	(3, CuDevice(2))
      From worker 2:	(2, CuDevice(1))


Lets run our benchmark in parallel across all GPUs:

In [39]:
let
    carr = cu(rand(10_000_000))
    pmap(WorkerPool(procs()), 1:4) do i
        @btime CUDA.@sync sin.($carr) .+ 1
    end
end

  84.954 μs (37 allocations: 1.91 KiB)
      From worker 4:	  81.557 μs (37 allocations: 1.91 KiB)
      From worker 3:	  81.547 μs (37 allocations: 1.91 KiB)
      From worker 2:	  81.196 μs (37 allocations: 1.91 KiB)


4-element Vector{CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}}:
 Float32[1.730833, 1.1198826, 1.3534169, 1.7853978, 1.5614758, 1.4536244, 1.4328939, 1.367788, 1.5307729, 1.6284117  …  1.2086698, 1.7418078, 1.5088024, 1.6645186, 1.3867525, 1.2837766, 1.7777662, 1.512737, 1.6256388, 1.442714]
 Float32[1.730833, 1.1198826, 1.3534169, 1.7853978, 1.5614758, 1.4536244, 1.4328939, 1.367788, 1.5307729, 1.6284117  …  1.2086698, 1.7418078, 1.5088024, 1.6645186, 1.3867525, 1.2837766, 1.7777662, 1.512737, 1.6256388, 1.442714]
 Float32[1.730833, 1.1198826, 1.3534169, 1.7853978, 1.5614758, 1.4536244, 1.4328939, 1.367788, 1.5307729, 1.6284117  …  1.2086698, 1.7418078, 1.5088024, 1.6645186, 1.3867525, 1.2837766, 1.7777662, 1.512737, 1.6256388, 1.442714]
 Float32[1.730833, 1.1198826, 1.3534169, 1.7853978, 1.5614758, 1.4536244, 1.4328939, 1.367788, 1.5307729, 1.6284117  …  1.2086698, 1.7418078, 1.5088024, 1.6645186, 1.3867525, 1.2837766, 1.7777662, 1.512737, 1.6256388, 1.442714]

Note, `carr` was defined and moved to GPU on the master process. Julia automatically sent it to the worker GPUs, then automatically sent the results back to the master GPU. 

In doing so, the array passed through CPU memory, so its not the most efficient (but its the easiest).

To go straight GPU-to-GPU, you can use _unified memory_ on a single-node, or CUDA MPI transport (later this talk).

## Multi-GPU (multiple nodes, elastic)

In [40]:
using ClusterManagers

In [41]:
em = ElasticManager(
    # Perlmutter specific ↓
    addr = IPv4(first(filter(!isnothing, match.(r"inet (.*)/.*hsn0", readlines(`ip a show`)))).captures[1]),
    port = 0
);

In [62]:
em

ElasticManager:
  Active workers : [ 5,6,7,8,9,10,11,12]
  Number of workers to be added  : 0
  Terminated workers : []
  Worker connect command : 
    /global/u1/m/marius/.julia/juliaup/julia-1.9.3+0.x64.linux.gnu/bin/julia --project=/global/u1/m/marius/work/gpu_science_day_julia/Project.toml -e 'using ClusterManagers; ClusterManagers.elastic_worker("JUYPFO3Hwb8flWSH","10.249.6.77",39653)'

Now submit a job, e.g. with:
```bash
salloc -C gpu -q regular -t 00:30:00 --cpus-per-task 32  --gpus-per-task 1 --ntasks-per-node 4 --nodes 8 -A mp107
```
then run the "worker connect command" printed above (could also do all-in-one as a batch job).

With more GPUs across different nodes, its more complex to assign one unique GPU to each process. Instead we can use this utility function:

In [76]:
using CUDADistributedTools

In [77]:
CUDADistributedTools.assign_GPU_workers()

┌ Info: Processes (4):
│  (myid = 1, host = nid001293, device = CuDevice(0): NVIDIA A100-SXM4-40GB 1c40175b))
│  (myid = 2, host = nid001293, device = CuDevice(1): NVIDIA A100-SXM4-40GB f179efe2))
│  (myid = 3, host = nid001293, device = CuDevice(2): NVIDIA A100-SXM4-40GB 36d32866))
└  (myid = 4, host = nid001293, device = CuDevice(3): NVIDIA A100-SXM4-40GB 634451b9))


Let's run parallel benchmarks again:

In [78]:
@everywhere using CUDA, BenchmarkTools

In [79]:
let
    carr = cu(rand(10_000_000))
    pmap(WorkerPool(procs()), 1:nprocs()) do i
        @btime CUDA.@sync sin.($carr) .+ 1
        return nothing
    end
end;

  85.044 μs (37 allocations: 1.91 KiB)
      From worker 2:	  81.798 μs (37 allocations: 1.91 KiB)
      From worker 3:	  81.747 μs (37 allocations: 1.91 KiB)
      From worker 4:	  82.018 μs (37 allocations: 1.91 KiB)


## Multi-GPU (multiple nodes, MPI)

Installing MPI for Julia and configuring:

```julia
pkg> add MPI MPIPreferences

julia> MPIPreferences.use_system_binary(;vendor="cray", mpiexec="srun") # <- options are Perlmutter specific

┌ Info: MPI implementation identified
│   libmpi = "libmpi_gnu_91.so"
│   version_string = "MPI VERSION    : CRAY MPICH version 8.1.25.17 (ANL base 3.4a2)\nMPI BUILD INFO : Sun Feb 26 15:15 2023 (git hash aecd99f)\n"
│   impl = "CrayMPICH"
│   version = v"8.1.25"
└   abi = "MPICH"
┌ Info: MPIPreferences changed
│   binary = "system"
│   libmpi = "libmpi_gnu_91.so"
│   abi = "MPICH"
│   mpiexec = "srun"
│   preloads =
│    1-element Vector{String}:
│     "libmpi_gtl_cuda.so"
└   preloads_env_switch = "MPICH_GPU_SUPPORT_ENABLED"
```

(This works thanks to among others NERSC's Johannes Blaschke's contributions to MPI.jl) 

You can put SLURM script and Julia script in one file 
`test_script.jl`:

```julia
#!/bin/bash
#SBATCH -C gpu -q regular -A mp107
#SBATCH -t 00:05:00 
#SBATCH --cpus-per-task 32 --gpus-per-task 1 --ntasks-per-node 4 --nodes 4
#=
srun /global/u1/m/marius/.julia/juliaup/julia-1.9.3+0.x64.linux.gnu/bin/julia $(scontrol show job $SLURM_JOBID | awk -F= '/Command=/{print $2}')
exit 0
# =#

using MPIClusterManagers, Distributed, CUDA
mgr = MPIClusterManagers.start_main_loop(MPIClusterManagers.MPI_TRANSPORT_ALL)

let
    carr = cu(rand(10_000_000))
    pmap(WorkerPool(procs()), 1:nprocs()) do i
        @btime CUDA.@sync sin.($carr) .+ 1
    end
end

MPIClusterManagers.stop_main_loop(mgr)
```

Then `sbatch test_script.jl`.

Here, movement of memory between GPUs will happen via CUDA MPI transport 🚀

## Multi-GPU (multiple nodes, MPI, notebooks)

### Some code in a notebook:

In [ ]:
let
    carr = cu(rand(10_000_000))
    pmap(WorkerPool(procs()), 1:nprocs()) do i
        @btime CUDA.@sync sin.($carr) .+ 1
        return nothing
    end
end;

### Now use:

In [ ]:
using ParameterizedNotebooks

In [ ]:
nb = ParameterizedNotebook("talk.ipynb", sections=("Some code in a notebook:",))

In [ ]:
nb()

You can put the call to the notebook code directly in a `test_script_2.jl`:
```julia
#!/bin/bash
#SBATCH -C gpu -q regular -A mp107
#SBATCH -t 00:05:00 
#SBATCH --cpus-per-task 32 --gpus-per-task 1 --ntasks-per-node 4 --nodes 4
#=
srun /global/u1/m/marius/.julia/juliaup/julia-1.9.3+0.x64.linux.gnu/bin/julia $(scontrol show job $SLURM_JOBID | awk -F= '/Command=/{print $2}')
exit 0
# =#

using MPIClusterManagers, Distributed, CUDA
mgr = MPIClusterManagers.start_main_loop(MPIClusterManagers.MPI_TRANSPORT_ALL)

nb = ParameterizedNotebook("talk.ipynb", sections=("Some code in a notebook:",))
nb()

MPIClusterManagers.stop_main_loop(mgr)
```

With some care in the organization of your sections, you can iterate on code in the notebook, even test it in parallel using on-the-fly `ElasticManager` workers, then submit the identical code as an MPI job for large-scale runs 🎉

## Wishlist

* More robust and easier CUDA.jl task/threading support
* An easy way to use MPI CUDA transport protocol from within Jupyter jobs
* A _multi-node_ GPU monitor, even just a command-line one
    * `nvitop`, `btop` (PR), and `gpustat` are some good command line single-node options